# Práctico 4:

1. Implementar un pipeline de clasificación con la muestra que se vió en clase. El código para traer el dataset es el siguiente:

In [6]:
from sklearn.datasets import fetch_openml

# Obtencion y carga de datos
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()
dict_keys = (['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])
X, y = mnist["data"], mnist["target"]
print(X)
print(y)

/home/fran/Documentos/5 AÑO/Sistemas Inteligentes carpeta/Sistemas-Inteligentes/Clase 4/my_env/lib/python3.8/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
['5' '0' '4' ... '4' '5' '6']


In [16]:
# Escalamos todos los datos a 0 1 (Normalización)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X = scaler.fit_transform(X)

In [17]:
# Tenemos que hacer un modelo que detecte cuando hay un 5

from sklearn.model_selection import train_test_split

# CREAR DATASET DE EVALUACION

train_set_X, test_set_X, = train_test_split(X, test_size=0.2, random_state=42)
train_set_y, test_set_y = train_test_split(y, test_size=0.2, random_state=42)

print(train_set_X, test_set_X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
# Filtramos las etiquetas (y) para que detecten los 5

train_set_y_5 = (train_set_y == 5)
test_set_y_5 = (test_set_y == 5)

print(list(filter(lambda X: X, test_set_y_5.tolist())))


[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [20]:
# Clasificador binario

from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(train_set_X, train_set_y_5)


SGDClassifier(random_state=42)

In [ ]:
# Vemos si predice correctamente o que tan bien lo hace
sgd_clf.predict(X[11].reshape(1, -1))

array([ True])

In [22]:
# Vemos la accuracy con Validacion Cruzada

from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, train_set_X, train_set_y_5, cv=3, scoring="accuracy")

array([0.97712541, 0.97519687, 0.97203472])

In [23]:
# Medimos accuracy en Dummy Classifier

from sklearn.dummy import DummyClassifier

never_5_clf = DummyClassifier(strategy="most_frequent")

cross_val_score(never_5_clf, train_set_X, train_set_y_5, cv=3, scoring="accuracy")

array([0.91000161, 0.91000161, 0.90999679])

In [24]:
# Matriz de confusion

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

train_set_y_pred = cross_val_predict(sgd_clf, train_set_X, train_set_y_5, cv=3)
confusion_matrix(train_set_y_5, train_set_y_pred)

array([[50526,   434],
       [  978,  4062]])

In [25]:
# Precision y recall

from sklearn.metrics import precision_score, recall_score

# Precision
p = precision_score(train_set_y_5, train_set_y_pred)
print("Precision:", p)
# Recall
r = recall_score(train_set_y_5, train_set_y_pred)
print("Recall:", r)

Precision: 0.9034697508896797
Recall: 0.805952380952381


In [26]:
# F1 score, precision y recall en una sola metrica

from sklearn.metrics import f1_score

f1_score(train_set_y_5, train_set_y_pred)


0.8519295302013422

In [28]:
#precision/recall trade off curve
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_curve, f1_score, auc


y_scores = sgd_clf.decision_function

ValueError: Found input variables with inconsistent numbers of samples: [70000, 56000]

2. Luego conteste las siguientes preguntas:
- ¿Que es un clasificador Binario?
- ¿Por qué un modelo con gran accuracy puede que no ser un buen modelo?
- ¿Por qué existe un trade off entre presicion y recall?
- ¿F1 es la mejor métrica que se puede utilizar al momento de optimizar un modelo? Justifique.
- ¿Para que sirve la curva ROC?
3. Utilice el clasificador KNeighborsClassifier y optimice los hiperparámetros utilizando GridSearch. ¿Qué accuracy se alcanza en el test set?